# Book Review Data

Imports a sample of 3M Amazon book reviews of 212K unique books.
The original dataset can be found on Kaggle at:

https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews

In [1]:
%load_ext autoreload
%autoreload 2

import time

import numpy as np
import pandas as pd
import sqlite3 as sql

import sql_ingest as ingest
%run ../query/search.py # importing as python package does not work because it requires navigating up the file hierarchy, which is not supported by Python

## Load Data
First, let's load the raw data from CSV and inspect it:

In [2]:
# Prerequisite: download the original dataset from Kaggle and place it in data/raw
# https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews

book_reviews = pd.read_csv('../raw/Books_rating.csv') # 3GB @ 20sec = 150 MB/sec
book_reviews.head(2)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...


In [3]:
book_reviews.shape

(3000000, 10)

In [4]:
book_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


Inspecting missing data shows that about 20% of reviews are lacking a user. Also, a negligible but nonzero numer of reviews are lacking review text, review title, and/or product title.

In [5]:
# Note: about 20% of reviews (over half a million) lack a user
# A negligible but nonzero number of reviews lack product title, review text, or review title.
book_reviews.isnull().sum()

Id                          0
Title                     208
Price                 2518829
User_id                561787
profileName            561905
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64

In [6]:
# 3M records @ 14sec = 214K records/sec
book_reviews.dropna(subset = ['Title', 'review/summary', 'review/text'], inplace = True)

In [7]:
book_data = pd.read_csv('../raw/books_data.csv') # 180 MB @ 1.7s = 106 MB/sec (150K records/sec)
book_data.head(2)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN


In [8]:
book_data.shape

(212404, 10)

In [10]:
book_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [11]:
book_data.isnull().sum()

Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64

In [12]:
book_data[book_data.Title.isna()]

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
1066,NaN,NaN,['Maharshi Ramana'],http://books.google.com/books/content?id=uq0uj...,http://books.google.nl/books?id=uq0ujwEACAAJ&d...,NaN,2015-12-15,http://books.google.nl/books?id=uq0ujwEACAAJ&d...,NaN,1.0


In [13]:
book_data.dropna(subset = ['Title'], inplace = True)

### Fix Missing Product IDs

There is a problem in the book data: it is lacking a product_id with which to join review data. Reviews include both product ID and title, so let's fill in the missing product ID by using review data.

Although it is technically possible to join on book title for this particular dataset, our backend database rightfully expects each book/product to have a proper ID instead.

In [14]:
# 3M records @ 2.7sec = 1.1M records/sec (!). Note that the nature of the aggregate skips most records.
book_ids_and_titles = book_reviews.rename(columns = {'Id': 'product_id'}).groupby('Title')['product_id'].first()
pd.DataFrame(book_ids_and_titles)

,product_id
Title,
""" Film technique, "" and, "" Film acting """,B0000CK3SI
""" We'll Always Have Paris"": The Definitive Guide to Great Lines from the Movies",B000K1C2HS
"""... And Poetry is Born ..."" Russian Classical Poetry",B000HEEPWS
"""A Titanic hero"" Thomas Andrews, shipbuilder",B00087XUJ8
"""A Truthful Impression of the Country"": British and American Travel Writing in China, 1880-1949",0472111973
...,...
with an everlasting love,B000NRY0YC
work and Motivation,B000PSN464
www.whitbread.org/book,078930287X


Good news - all reviews have associated product data and all products except 6 have an associated review from which we can derive the product ID:

In [15]:
original_review_count = len(book_reviews)
full_reviews = book_reviews.merge(book_data, on = 'Title', how = 'inner') # 3M reviews @ 6.8 sec = 441K records/sec.
print(f"{original_review_count - len(full_reviews)} of {original_review_count} reviews lack product data")
full_reviews.isnull().sum()

0 of 2999377 reviews lack product data


Id                          0
Title                       0
Price                 2518250
User_id                561757
profileName            561871
review/helpfulness          0
review/score                0
review/time                 0
review/summary              0
review/text                 0
description            639909
authors                390568
image                  540224
previewLink            330566
publisher              782269
publishedDate          354520
infoLink               330566
categories             551193
ratingsCount          1360525
dtype: int64

In [16]:
book_data_merged = book_data.join(book_ids_and_titles, on = 'Title', how = 'left')
book_data_merged.isnull().sum()

Title                 0
description       68441
authors           31413
image             52075
previewLink       23836
publisher         75885
publishedDate     25305
infoLink          23836
categories        41198
ratingsCount     162652
product_id            6
dtype: int64

A product without ID is unusable because we require an ID as a primary key. Luckily there are only 6 of them out of over 200K. Let's remove these anomalies.

In [17]:
book_data_merged = book_data_merged.rename(columns = {'product_id': 'id'}).dropna(subset = ['id']).set_index('id')
book_data = book_data_merged
book_data.head(2)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
id,,,,,,,,,,
1882931173,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
0826414346,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN


## Import Into SQL - Reviews

In [2]:
conn = sql.connect('../products.sql')

Before importing review data, we need to normalize the dataset to fit our database schema as follows:
- rename columns to fit DB schema
- convert 'x/y' format of review helpfulness into separate upvote/downvote numeric columns
- drop the Title column. It reflects the product title, which belongs in the product table
    - we can retrieve product title for a review on demand by joining with products on product_id
- drop the price column. It does not belong to the review table and is not relevant for this project.
    - price also does not belong in the product table. A price is associated with a product listing, not a product.

In [19]:

# Convert review data to a format that matches our database
def normalize_reviews(reviews: pd.DataFrame) -> pd.DataFrame:
    result = reviews.copy()
    def parse_helpfulness(helpfulness: str) -> tuple:
        parts = helpfulness.partition('/')
        upvotes = int(parts[0])
        downvotes = int(parts[2]) - upvotes
        return (upvotes, downvotes)
    up_down_votes = result['review/helpfulness'].map(parse_helpfulness)
    result['upvotes'] = pd.Series([t[0] for t in up_down_votes], index = reviews.index)
    result['downvotes'] = pd.Series([t[1] for t in up_down_votes], index = reviews.index)
    return result.drop(columns = ['review/helpfulness', 'Price', 'Title']).rename(columns = {
        'Id': 'product_id',
        'User_id': 'user_id',
        'profileName': 'user_name',
        'review/score': 'rating',
        'review/summary': 'title',
        'review/text': 'review',
        'review/time': 'timestamp'
    })

book_reviews_norm = normalize_reviews(book_reviews) # 3M records @ 7sec = 428K records / sec
book_reviews_norm

,product_id,user_id,user_name,rating,timestamp,title,review,upvotes,downvotes
0,1882931173,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,7,0
1,0826414346,A30TK6U7DNS82R,Kevin Killian,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,10,0
2,0826414346,A3UH4UZ4RSVO82,John Granger,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",10,1
3,0826414346,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",7,0
4,0826414346,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,3,0
...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,NaN,NaN,4.0,937612800,Difficult,"This is an extremely difficult book to digest,...",14,5
2999996,B000NSLVCU,A1SMUB9ASL5L9Y,jafrank,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...,1,0
2999997,B000NSLVCU,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i...",0,0
2999998,B000NSLVCU,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,...",1,10


In [ ]:
# 3M reviews @ 2m 38 sec ~ 19K reviews/sec
# Performance without indices: 3M reviews @ 50.9 sec = 59K reviews / sec
#   + 1m 18 sec to create indices after the fact
ingest.import_reviews(book_reviews_norm.drop(columns = ['user_name']), conn)

Inserting 2,999,377 records into review...
1.65: inserted 100,000 of 2,999,377 records (3.3%) @ 60702 records / sec
3.07: inserted 200,000 of 2,999,377 records (6.7%) @ 65096 records / sec
4.57: inserted 300,000 of 2,999,377 records (10.0%) @ 65637 records / sec
6.29: inserted 400,000 of 2,999,377 records (13.3%) @ 63631 records / sec
8.03: inserted 500,000 of 2,999,377 records (16.7%) @ 62229 records / sec
9.42: inserted 600,000 of 2,999,377 records (20.0%) @ 63716 records / sec
10.70: inserted 700,000 of 2,999,377 records (23.3%) @ 65402 records / sec
12.27: inserted 800,000 of 2,999,377 records (26.7%) @ 65192 records / sec
14.43: inserted 900,000 of 2,999,377 records (30.0%) @ 62381 records / sec
16.61: inserted 1,000,000 of 2,999,377 records (33.3%) @ 60202 records / sec
18.90: inserted 1,100,000 of 2,999,377 records (36.7%) @ 58207 records / sec
21.08: inserted 1,200,000 of 2,999,377 records (40.0%) @ 56935 records / sec
23.29: inserted 1,300,000 of 2,999,377 records (43.3%) @ 55

## Import Into SQL - Book Data

Before importing book data, we need to adapt it to our database schema as follows:
- rename columns to fit table schema
- convert list of author / genre to single genre
- drop images
- drop preview link
- create normalized seach strings for title + author

This phase focuses on populating the product table. A separate pass will handle many-to-many product metadata stored in other tables. This includes resolving multiple artists/genres for a product and restoring any relevant data dropped in this phase (e.g. images).

In [141]:
def normalize_books(books: pd.DataFrame) -> pd.DataFrame:
    result = books.rename(columns = {
        'Title': 'title',
        'authors': 'creator',
        'publishedDate': 'release_date',
        'infoLink': 'url',
        'image': 'images',
        'categories': 'subcategory'
    }).drop(columns = ['previewLink', 'ratingsCount'])
    result.index.name = 'id'
    result['creator'] = result.creator.map(ingest.get_single_value)
    result['creator_search'] = result.creator.map(search_text)
    result['title_search'] = result.title.map(search_text)
    result['category'] = 'Books'
    result['subcategory'] = result.subcategory.map(ingest.get_single_value)
    return result

books_normalized = normalize_books(book_data)
books_normalized.head(2)

,title,description,creator,images,publisher,release_date,url,subcategory,creator_search,title_search,category
id,,,,,,,,,,,
1882931173,Its Only Art If Its Well Hung!,NaN,Julie Strain,http://books.google.com/books/content?id=DykPA...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,Comics & Graphic Novels,juliestrain,itsonlyartifitswellhung,Books
0826414346,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,Philip Nel,http://books.google.com/books/content?id=IjvHQ...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,Biography & Autobiography,philipnel,drseussamericanicon,Books


In [ ]:
clean_book_data_import = False
if clean_book_data_import:
    conn.execute("DELETE FROM product WHERE category = 'Books'")
    print('Removed existing book data')

Removed existing book data


In [ ]:
# 212K insertions @ 1.5 sec = 141K insertions / sec
ingest.import_products(books_normalized, conn)

### Verify Results

Verify that we have book data and can access it in a reasonable timeframe:

In [7]:
print("Number of books:")
pd.read_sql_query("SELECT COUNT(*) AS books FROM product WHERE category = 'Books'", conn) # instant results

Number of books:


,books
0,212397


In [9]:
print("Number of reviews:")
pd.read_sql_query("SELECT COUNT(*) AS book_reviews FROM review r JOIN product p on r.product_id = p.id WHERE category = 'Books'", conn) # fast (0.4sec) results

Number of reviews:


,book_reviews
0,2597367


In [10]:
print("Book samples:")
pd.read_sql_query("SELECT * FROM product WHERE category = 'Books' LIMIT 5", conn) # instant results

Book samples:


,id,title,title_search,creator,creator_search,publisher,description,category,subcategory,release_date
0,0001047604,Aurora Leigh,auroraleigh,Elizabeth Barrett Browning,elizabethbarrettbrowning,None,Being an undated and detailed autograph manusc...,Books,English poetry,1857
1,0001047736,Summer of the Danes,summerofdanes,nan,nan,None,None,Books,nan,None
2,0001047825,Potters Field Audio,pottersfieldaudio,"Ellis Peters, Derek Jacobi",derekjacobiellispeters,None,None,Books,nan,19??
3,0001047876,Life of Samuel Johnson,lifeofsamueljohnson,James Boswell,jamesboswell,BEYOND BOOKS HUB,"Originally published in 1791, this biography o...",Books,Biography & Autobiography,2021-01-01
4,0001048228,Pale Battalions,palebattalions,Robert Goddard,robertgoddard,Delta,"Six months after her husband's sudden death, L...",Books,Fiction,2007-05-29


In [ ]:
# Genres by size: Instant results. Observations:
# About 10K = 5% of books are in a singular genre due to misclassification
# About 40K = 20% of books have a missing/nan genre
# -> About 25% of books do not have an actionable genre
print("Book genres:")
pd.read_sql_query("""
SELECT subcategory AS genre, COUNT(*) AS books
FROM product
WHERE category = 'Books'
GROUP BY  subcategory ORDER BY books DESC
""", conn)

Book genres:


,genre,books
0,nan,41196
1,Fiction,23419
2,Religion,9458
3,History,9329
4,Juvenile Fiction,6643
...,...,...
10879,adventure-juvenile fiction,1
10880,al-Hizb al Suri al-Qawni al-Ijtima'i,1
10881,juvenile romance,1
10882,Ātman,1


In [ ]:
print ('Most Reviewed Books:')
pd.read_sql_query("""
SELECT p.title, p.creator AS author, COUNT(*) as reviews 
FROM product p JOIN review r ON r.product_id = p.id 
WHERE p.category = 'Books'
GROUP BY p.title, p.creator ORDER BY reviews DESC LIMIT 10
""", conn)

Most Reviewed Books:


,title,author,reviews
0,Harry Potter and The Sorcerer's Stone,J. K. Rowling,6796
1,The Hobbit There and Back Again,John Ronald Reuel Tolkien,4437
2,"The Hobbit; Or, There and Back Again",nan,4426
3,The Hobbit,J. R. R. Tolkien,4425
4,"The Hobbitt, or there and back again; illustra...",J. R. R. Tolkien,4419
5,The Hobbit or There and Back Again,J. R. R. Tolkien,4372
6,The Giver,Lois Lowry,3822
7,Holes,Louis Sachar,3567
8,The Catcher in the Rye,nan,3280
9,The Catcher in the Rye [Audiobook] [Cd] [Unabr...,John Donovan,3270


In [14]:
print ('Most Reviewed Authors:')
pd.read_sql_query("""
SELECT p.creator AS author, COUNT(*) as reviews 
FROM product p JOIN review r ON r.product_id = p.id 
WHERE p.category = 'Books'
GROUP BY p.creator ORDER BY reviews DESC LIMIT 10
""", conn)

Most Reviewed Authors:


,author,reviews
0,nan,350723
1,J. R. R. Tolkien,19629
2,Charles Dickens,14150
3,John Steinbeck,12199
4,John Ronald Reuel Tolkien,11633
5,Jane Austen,10325
6,J. K. Rowling,9976
7,Harper Lee,9607
8,Stephen King,8990
9,George Orwell,8280


Notice that most results are title variations on the same book. So for a robust search/recommendation engine, we need to disambiguate between multiple variations of the same product.

### Book Metadata
To do: insert additional product metadata in tables outside of products. For example, to handle books that have multiple genres or authors.

In [86]:
def expand_multiple_values_in_coluimn(data: pd.DataFrame, multi_value_column: str, id_column = 'id') -> pd.DataFrame:
    """"
    Given the name of a column containing multiple values, generate a unique id/value pair for each value in each multi-value column.
    The intended use case is to handle products that have multiple entries in a column like author or genre that is usually but not always single-value.

    Example input:
    [
      { 'id': 'a', 'genre': ['Synthwave', 'Synthpop'] },
      { 'id': 'b', 'genre': [] },
      { 'id': 'c', 'genre': 'Pop' }
    ]

    Example output:
    [
        { 'id': 'a', 'genre', 'Sythwave' },
        { 'id': 'a', 'genre', 'Sythpop' },
        { 'id': 'c', 'genre', 'Pop' }
    ]
    """
    # to do: implement this
    return data 

### Close
Don't forget to clean up the DB connection

In [89]:
conn.close()